In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from peft import PeftModel

/home/wangrui/anaconda3/envs/hugf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 加载基础模型

In [2]:
model = AutoModelForCausalLM.from_pretrained("/data/jasonzhou/Models/Qwen1.5-0.5B-Chat")
tokenizer = AutoTokenizer.from_pretrained("/data/jasonzhou/Models/Qwen1.5-0.5B-Chat")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 加载Lora模型

In [3]:
p_model = PeftModel.from_pretrained(model, model_id="/data/jasonzhou/Chatbot/INFH5000/train/models/checkpoint-4000")
p_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-19): 20 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (rotary_emb): Qwen2RotaryEmbedding()
            )
            (mlp): Qwen2MLP(
              (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
              (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
              (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm)

In [4]:
ipt = tokenizer("Human: {}\n{}".format("下雨天膝盖痛是什么原因？", "").strip() + "\n\nAssistant: ", return_tensors="pt")
# tokenizer.decode(p_model.generate(**ipt, do_sample=False)[0], skip_special_tokens=True)
tokenizer.decode(p_model.generate(**ipt, max_length=128)[0])

'Human: 下雨天膝盖痛是什么原因？\n\nAssistant: 膝盖是人体的重要关节，当下雨时，由于路面湿滑、雨水进入，容易导致膝关节受损。此外，如果膝盖受凉或者长时间保持一种姿势，也可能加重膝盖疼痛。建议在下雨天注意保暖，避免膝盖受到伤害。<|im_end|>'

## 模型合并

In [10]:
merge_model = p_model.merge_and_unload()
merge_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(i

In [11]:
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt")
tokenizer.decode(merge_model.generate(**ipt, do_sample=False)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 1. 熟悉考试大纲'

## 完整模型保存

In [9]:
merge_model.save_pretrained("./chatbot/merge_model")